# Gold Layer Smoke Checks
This notebook validates tower_key propagation from Silver to Gold and basic row presence.

In [ ]:
# Setup
catalog = 'telkom_analytics'
spark.sql(f'USE CATALOG {catalog}')

## 1) Silver: Verify tower_sk presence in source tables

In [ ]:
def has_col(table, col_name):
    try:
        return col_name in spark.table(f'{catalog}.{table}').columns
    except Exception:
        return False
tables = ['customer_usage','customer_feedback','voice_transcriptions','load_shedding_schedules']
{t: has_col(t, 'tower_sk') for t in tables}

## 2) Gold: Check tower_key exists and has non-null rows

In [ ]:
gold_tables = [
  'fact_customer_usage',
  'fact_customer_feedback',
  'fact_maintenance_activity',
  'fact_network_performance'
]
results = {}
for t in gold_tables:
    try:
        df = spark.table(f'{catalog}.{t}')
        results[t] = {
            'has_tower_key': 'tower_key' in df.columns,
            'non_null_tower_key_rows': df.filter('tower_key IS NOT NULL').limit(1).count()
        }
    except Exception as e:
        results[t] = {'error': str(e)}
results

## 3) Join consistency: tower_key -> dim_tower

In [ ]:
try:
    fcu = spark.table(f'{catalog}.fact_customer_usage')
    dt = spark.table(f'{catalog}.dim_tower').filter('is_current = true')
    cnt = fcu.join(dt, 'tower_key', 'inner').limit(1).count()
    {'join_ok': cnt > 0}
except Exception as e:
    {'error': str(e)}

## 4) New aggregate: agg_hourly_usage_by_tower row count

In [ ]:
try:
    cnt = spark.table(f'{catalog}.agg_hourly_usage_by_tower').count()
    {'row_count': cnt}
except Exception as e:
    {'error': str(e)}